<a href="https://colab.research.google.com/github/r-y-a-n-m-a-r-t-i-n/CIS5190_Project/blob/main/Project_Amazon_Fine_Food_Reviews_jbr_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Some imports that you might need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader, TensorDataset
import nltk
import os

# Switch PyTorch to GPU compute
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


Hi Jack

For the data, you can click download at
https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

You can choose to store the data in your google drive and mount your drive for accessing them from the following cell.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Or you can download it by the following cell:

In [ ]:
!pip install --upgrade --no-cache-dir gdown
if not os.path.exists("Reviews.csv"):
    !gdown 1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Downloading...
From (uriginal): https://drive.google.com/uc?id=1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk
From (redirected): https://drive.google.com/uc?id=1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk&confirm=t&uuid=54c01a51-147b-4239-aa96-d11bdbf8e6f7
To: /content/Reviews.csv
100% 301M/301M [00:05<00:00, 52.4MB/s]


In [ ]:
# Remeber to change the path here for the corresponding files you need
df = pd.read_csv('Reviews.csv')
print(df.shape)
df = df
print(df.shape)

(568454, 10)
(568454, 10)


In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
# import libraries for preprocessing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.sentiment import SentimentAnalyzer
from sklearn.preprocessing import MultiLabelBinarizer
from string import punctuation
from string import digits
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# function to remove stop words
eng_stopwords = stopwords.words('english') 
def filter_words(alist):
  ret_l = []
  for w in alist:
    for el in w:
        if el in punctuation:
          w = w.replace(el,"")
    if w.lower() not in eng_stopwords and w not in punctuation and w != '...' and w !='br' and w not in digits and w != 'nt':
      ret_l.append(w.lower())
  return ret_l

In [ ]:
# preprocess data

df['tokenized_text'] = df['Text'].apply(word_tokenize)
df['filtered_text'] = df['tokenized_text'].apply(filter_words)
all_reviews = df['filtered_text'].tolist()
# print(all_reviews)
all_words=[element for sublist in all_reviews for element in sublist]
total_words=len(all_words)
count_words = Counter(all_words)
sorted_words=count_words.most_common(total_words)

# print(sorted_words[:10])

In [ ]:
# integer encode the words
sorted_words = sorted_words[:20000]
vocab_to_int={w:i+1 for i,(w,c) in enumerate(sorted_words)}
print(vocab_to_int)


{'like': 1, 'good': 2, 'one': 3, 'taste': 4, 'great': 5, 'coffee': 6, 'product': 7, 'flavor': 8, 'tea': 9, 'would': 10, 'love': 11, 'food': 12, 'get': 13, 'really': 14, 'much': 15, 'also': 16, 'amazon': 17, 'use': 18, 'time': 19, 'little': 20, 'buy': 21, 'tried': 22, 'best': 23, 'price': 24, 'even': 25, 'find': 26, 'make': 27, 'well': 28, 'better': 29, 'dog': 30, 'try': 31, 'eat': 32, 'first': 33, 'water': 34, 'chocolate': 35, 'bag': 36, 'could': 37, 'used': 38, 'sugar': 39, 'found': 40, 'cup': 41, 'drink': 42, 'sweet': 43, 'made': 44, 'box': 45, 'bought': 46, 'way': 47, 'think': 48, 'two': 49, 'day': 50, 'since': 51, 'tastes': 52, 'still': 53, 'store': 54, 'go': 55, 'order': 56, 'know': 57, 'got': 58, 'bit': 59, 'give': 60, 'recommend': 61, 'many': 62, 'nice': 63, 'delicious': 64, 'favorite': 65, 'want': 66, 'flavors': 67, 'never': 68, 'mix': 69, 'something': 70, 'every': 71, 'free': 72, 'ca': 73, 'hot': 74, 'dogs': 75, 'brand': 76, 'cat': 77, 'stuff': 78, 'always': 79, 'without': 80,

In [ ]:
# transform reviews into lists of numbers
e_rev = []
for review in all_reviews:
  e = []
  for word in review:
    if word not in vocab_to_int.keys():
      e.append(0)
    else:
      e.append(vocab_to_int[word])
  e_rev.append(e)

In [ ]:
# make the feature array and make the inputs all the same length
length = 100
feats=np.zeros((len(e_rev), length), dtype=int)

for i, rev in enumerate(e_rev):
  if len(rev) > length:
    feats[i,:] = np.array(rev[:length])
  else:
    for j in range(len(rev)):
      feats[i,j] = rev[j]
# X_train
feats

array([[  46,  202, 4707, ...,    0,    0,    0],
       [   7,  247, 2033, ...,    0,    0,    0],
       [6888,  168, 9171, ...,    0,    0,    0],
       ...,
       [ 250,   99,   60, ...,    0,    0,    0],
       [  23,   83,  711, ...,    0,    0,    0],
       [ 843,    7, 1298, ...,    0,    0,    0]])

In [ ]:
# creature training
df['Score']
labels = df['Score'].array
# y_train
# labels = labels / 5

# for l in range(len(labels)):
#   if labels[l] < 4:
#     labels[l] = 0
#   else:
#     labels[l] = 1

# labels = labels.to_numpy()

print(labels.shape)
print(labels)
print(feats.shape)
# feats = feats.T


(568454,)
<PandasArray>
[5, 1, 4, 2, 5, 4, 5, 5, 5, 5,
 ...
 5, 5, 2, 5, 5, 5, 2, 5, 5, 5]
Length: 568454, dtype: int64
(568454, 100)


In [ ]:
# Split into Train, Validation, Test



train_x=feats[:int(0.8*len(feats))]
train_y=labels[:int(0.8*len(feats))]
valid_x=feats[int(0.8*len(feats)):int(0.9*len(feats))]
valid_y=labels[int(0.8*len(feats)):int(0.9*len(feats))]
test_x=feats[int(0.9*len(feats)):]
test_y=labels[int(0.9*len(feats)):]
print(train_x.shape, valid_y.shape, test_y.shape)

In [ ]:
print(train_x, train_y)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=10000, multi_class='multinomial').fit(train_x, train_y)
# 1 - sum(abs(clf.predict(test_x) - test_y)) / len(test_y)


In [ ]:
print(np.unique(clf.predict(test_x)))
print(test_y)

In [ ]:
from sklearn import tree
clf3 = tree.DecisionTreeClassifier()
clf3 = clf3.fit(train_x, train_y)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(test_y, clf.predict(test_x)))
print(accuracy_score(test_y, clf3.predict(test_x)))

In [ ]:
df['Score']
df = pd.read_csv('Reviews.csv')
print(df.shape)
size = df.shape[0]
labels = df['Score'].array
labels

In [ ]:
train_x=feats[:int(0.8*size)]
train_y=labels[:int(0.8*size)]
valid_x=feats[int(0.8*size):int(0.9*size)]
valid_y=labels[int(0.8*size):int(0.9*size)]
test_x=feats[int(0.9*size):]
test_y=labels[int(0.9*size):]
print(train_x.shape, valid_y.shape, test_x.shape, test_y.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression().fit(train_x, train_y)
y_pred = clf.predict(test_x)
y_pred.shape

In [ ]:
sum = 0
for i in range(len(y_pred)):
  sum += abs(y_pred[i] - test_y[i])
score = sum / len(y_pred)
score

In [ ]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
# df['sentiment'] = df['Text'].apply(SentimentIntensityAnalyzer.polarity_scores)
sentiment = []
senanal = SentimentIntensityAnalyzer()
for sen in df['Text'].to_list():
  sentiment.append(senanal.polarity_scores(sen)['compound'])

In [ ]:
sentiment_mapped = []
for val in sentiment:
  sentiment_mapped.append(val*2 + 3)
# sentiment_mapped

In [ ]:
sum = 0
for i in range(len(sentiment_mapped)):
  sum += abs(sentiment_mapped[i] - df['Score'].tolist()[i])
score = sum / len(sentiment_mapped)
score

In [ ]:
# Dataset Shift
df = pd.read_csv('Reviews.csv')
print(df.shape)
df = df
print(df.shape)

In [ ]:
df.head()

In [ ]:
print(df['HelpfulnessNumerator'].unique())
print(df['HelpfulnessNumerator'].unique().shape)

In [ ]:
shift_df_train = df[df['HelpfulnessNumerator'] == 0]
shift_df_test = df[df['HelpfulnessNumerator'] != 0]


In [ ]:
shift_df_train['tokenized_text'] = shift_df_train['Text'].apply(word_tokenize)
shift_df_train['filtered_text'] = shift_df_train['tokenized_text'].apply(filter_words)
shift_all_reviews = shift_df_train['filtered_text'].tolist()
shift_df_test['tokenized_text'] = shift_df_test['Text'].apply(word_tokenize)
shift_df_test['filtered_text'] = shift_df_test['tokenized_text'].apply(filter_words)
shift_all_reviews_test = shift_df_test['filtered_text'].tolist()
# print(all_reviews)
shift_all_words=[element for sublist in shift_all_reviews for element in sublist]
shift_total_words=len(shift_all_words)
shift_count_words = Counter(shift_all_words)
shift_sorted_words=shift_count_words.most_common(shift_total_words)

In [ ]:
# integer encode the words
shift_sorted_words = shift_sorted_words[:20000]
shift_vocab_to_int={w:i+1 for i,(w,c) in enumerate(shift_sorted_words)}
print(shift_vocab_to_int)
shift_df_train.shape

In [ ]:
# transform reviews into lists of numbers
e_rev = []
for review in shift_all_reviews:
  e = []
  for word in review:
    if word not in shift_vocab_to_int.keys():
      e.append(0)
    else:
      e.append(shift_vocab_to_int[word])
  e_rev.append(e)

# transform reviews into lists of numbers
e_rev_test = []
for review in shift_all_reviews_test:
  e = []
  for word in review:
    if word not in shift_vocab_to_int.keys():
      e.append(0)
    else:
      e.append(shift_vocab_to_int[word])
  e_rev_test.append(e)

In [ ]:
length = 100
shift_feats=np.zeros((len(e_rev), length), dtype=int)
shift_feats_test=np.zeros((len(e_rev_test), length), dtype=int)

for i, rev in enumerate(e_rev):
  if len(rev) > length:
    shift_feats[i,:] = np.array(rev[:length])
  else:
    for j in range(len(rev)):
      shift_feats[i,j] = rev[j]

for i, rev in enumerate(e_rev_test):
  if len(rev) > length:
    shift_feats_test[i,:] = np.array(rev[:length])
  else:
    for j in range(len(rev)):
      shift_feats_test[i,j] = rev[j]

# X_train
print(shift_feats.shape)
shift_feats_test.shape

In [ ]:
# train_x=feats[:int(0.8*len(feats))]
# train_y=labels[:int(0.8*len(feats))]
# valid_x=feats[int(0.8*len(feats)):int(0.9*len(feats))]
# valid_y=labels[int(0.8*len(feats)):int(0.9*len(feats))]
# test_x=feats[int(0.9*len(feats)):]
# test_y=labels[int(0.9*len(feats)):]
# print(train_x.shape, valid_y.shape, test_y.shape)
# creature training
shift_df_train['Score']
shift_labels =shift_df_train['Score'].array
shift_labels_test =shift_df_test['Score'].array

# y_train
# labels = labels / 5

# for l in range(len(shift_labels)):
#   if shift_labels[l] < 3:
#     shift_labels[l] = 0
#   else:
#     shift_labels[l] = 1

# labels = labels.to_numpy()

print(shift_labels.shape)
print(shift_labels)
print(shift_feats.shape)
# feats = feats.T

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=10000, multi_class='multinomial').fit(shift_feats, shift_labels)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("training accuracy: " + str(accuracy_score(shift_labels, clf.predict(shift_feats))))
print("testing_accuracy: "+ str(accuracy_score(shift_labels_test, clf.predict(shift_feats_test))))